# Virtual flight over image

In [ ]:
from moviepy.editor import ImageClip
import moviepy.video.fx.all as vfx

from skimage import transform
import numpy as np

In [ ]:
 # function borrowed from https://zulko.github.io/moviepy/examples/star_worms.html
def trapzWarp(pic,cx,cy):
    """ Complicated function (will be latex packaged as a fx) """
    Y,X = pic.shape[:2]
    src = np.array([[0,0],[X,0],[X,Y],[0,Y]])
    dst = np.array([[cx*X,cy*Y],[(1-cx)*X,cy*Y],[X,Y],[0,Y]])
    tform = transform.ProjectiveTransform()
    tform.estimate(src,dst)
    im = transform.warp(pic, tform.inverse, output_shape=(Y,X))
    return (im*255).astype('uint8')

In [ ]:
def fly(image_path, video_res=(1920, 1080), duration=20, perspective=0.15):
    """Creates a virtual flight video from an image file

    Parameters
    ----------
    image_path : str
        Path to the image file
    video_res: (int, int)
        Resolution (width, height) of the output video (default: (1920, 1080))
            wider video -> faster scrolling
            higher video -> slower scrolling
    duration : int
        Duration of the video in seconds: (default: 5)
            higher -> longer videos -> slower flight
            lower -> shorter videos -> faster flight
    perspective : float
        Perspective parameter (default: 0.2)
            0.0 -> No perspective transformation
            smaller -> less perspective transformation -> slower flight
            higher (max = 0.499) -> more perspective transformation -> faster flight
        
    Returns
    -------
    cropped
        A moviepy video

    """

    clip = ImageClip(image_path).set_duration(duration)
    
    out_width, out_height = video_res
    
    # Resize for correct width after cropping
    width = out_width / (1 - 2 * perspective)
    resized = clip.resize(width=width)
    
    # Scroll backwards
    pixel_to_scroll = resized.h - out_height
    speed = pixel_to_scroll / duration # pixel per second
    scrolling = vfx.scroll(resized, h=out_height, y_speed=speed)
    backwards = vfx.time_mirror(scrolling)
    
    # Add perspective
    fl_im = lambda pic : trapzWarp(pic, perspective, 0.0)
    warped = backwards.fl_image(fl_im)
    
    # Clip sides width no data
    clipping = width * perspective
    cropped = vfx.crop(warped, x1=clipping, width=out_width)
    
    return cropped

In [ ]:
image_path = 'image.tif'
mp4_path = 'video.mp4'

In [ ]:
fly(image_path, video_res=(1920, 1080), duration=20, perspective=0.15).write_videofile(str(mp4_path), fps=60)

## TODO
### Add logos

https://github.com/Zulko/moviepy/issues/127

In [ ]:
import moviepy.editor as mp

video = mp.VideoFileClip("video.mp4")

logo = (mp.ImageClip("logo.png")
          .set_duration(video.duration)
          .resize(height=50) # if you need to resize...
          .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
          .set_pos(("right","top")))

final = mp.CompositeVideoClip([video, logo])
final.write_videofile("test.mp4")